In [4]:
# Importing libraries
import os
import pandas as pd
import numpy as np
import pandas_ta as ta
import talib as ta2
import plotly.express as px
from datetime import date
import plotly.graph_objects as go
from yahooquery import Ticker

# yahooquery wokrs better than yfinance
# yahooquery project documentation: https://pypi.org/project/yahooquery/
# yfinance project documentation: https://pypi.org/project/yfinance/
# pandas_ta project documentation: https://twopirllc.github.io/pandas-ta/


In [5]:
# Define functions to retrieve market data and create graphs

def get_ticker(ticker):
    try:
        ticker = Ticker(ticker)
        return ticker
    except:
        print('Error: Ticker not found')
        return None

def get_price_hist(ticker, period, interval):
    hist = ticker.history(period=period, interval=interval)
    hist = hist.reset_index()
    # hist['date'] = pd.to_datetime(hist['date'])
    # hist['date'] = hist['date'].dt.strftime('%Y-%m-%d')
    return hist



def get_financials(df, col_name, metric_name):
    metric = df.loc[:, ['asOfDate', col_name]]
    metric_df = pd.DataFrame(metric).reset_index()
    metric_df.columns = ['Symbol', 'Year', metric_name]

    return metric_df


In [25]:

# Defining functions to detect patterns

def is_consolidating(df, percentage=2, num_candlesticks=15):
    recent_candlesticks = df[-num_candlesticks:]
    
    max_close = recent_candlesticks['close'].max()
    min_close = recent_candlesticks['close'].min()

    threshold = 1 - (percentage / 100)
    if min_close > (max_close * threshold):
        return True        

    return False

def is_breaking_out(df, percentage=2.5, num_candlesticks=15):
    last_close = df[-1:]['close'].values[0]

    if is_consolidating(df[:-1], percentage=percentage, num_candlesticks=num_candlesticks):
        recent_closes = df[-num_candlesticks + 1 :-1]

        if last_close > recent_closes['close'].max():
            return True

    return False



In [7]:
# Simple Plot for fundamental financial data
def line_chart(df, x, y, title):
    fig = px.line(df, x=x, y=y, template='simple_white',
                        title='<b>{} {}</b>'.format(name, title))

    return fig

def bar_chart(df, x, y, title):
    fig = px.bar(df, x=x, y=y, template='simple_white',
                        title='<b>{} {}</b>'.format(name, title))

    return fig



In [8]:

def plot_candlestick(df, slider=False, ma=False):
    # Create figure
    fig = go.Figure(data=[go.Candlestick(x=df['date'],
                                         open=df['open'],
                                         high=df['high'],
                                         low=df['low'],
                                         close=df['close'])]                                   
                    )
    # Add moving averages
    if ma:
        # Add the strategy to the DataFrame
        df.ta.strategy(mvg_avg)
        # Add EMA and SMA lines
        fig.add_trace(go.Scatter(x=df['date'], y=df['EMA_50'], line=dict(color='pink', width=1), name='EMA 50'))
        fig.add_trace(go.Scatter(x=df['date'], y=df['EMA_100'], line=dict(color='purple', width=1), name='EMA 100'))
        fig.add_trace(go.Scatter(x=df['date'], y=df['SMA_200'], line=dict(color='red', width=2), name='SMA 200'))
    else:
        pass
    # Change figure size
    fig.update_layout(
        autosize=True,
        width=1000,
        height=600
        )
    

    # Add title and xaxis
    fig.update_layout(
        title=f'{ticker_string.upper()} Stock Price',
        xaxis_rangeslider_visible=slider,
        xaxis_title='Date',
        yaxis_title='Price',
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    return fig


# Goal
================
* Iterate through a list of tickers
* Retrieve historical data from Yahoo Finance
* Determine range bound patterns in the data such as channels and triangles
* Check which tickers are in a Channel, Triangle or Wedge Patterns
* Determine likely breakout candidates
* Determine likely breakdown candidates
* Determine likely range bound candidates

Example Tickers:
================
* MYRG
* HAE
* LVMUY




Technical Indicators Definitions and Examples
================
https://library.tradingtechnologies.com/trade/chrt-technical-indicators.html

Chart Patterns
================
https://www.incrediblecharts.com/technical/chart_patterns.php


#  Average directional index (ADX)
ADX is used to quantify **trend strength**.

| ADX Value | Trend Strength           |
|-----------|-------------------------|
| 0-25      | Absent or Weak Trend     |
| 25-50     | Strong Trend             |
| 50-75     | Very Strong Trend        |
| 75-100    | Extremely Strong Trend   |


In [10]:
# Create a Strategy
mvg_avg = ta.Strategy(name="Dual_emas", ta=[{"kind": "ema", "length": 50}, {"kind": "ema", "length": 100}, {"kind": "sma", "length": 200}])

# Retrieve Historical Data from Yahoo Finance

In [18]:
# Get the current working directory
cwd = os.getcwd()

# Remove the last folder name from the current working directory to get the path to the parent folder
path = cwd[:-len(os.path.basename(cwd))]


# Define amount of data to retrieve
period_option = '5y'
interval_option = '1wk' # interval options: 1d, 1wk, 1mo

# # Get today's date
# today = date.today()

# # Enter company ticker. 
# ticker_string = input('Enter company ticker: ')

# # trim user input string
# ticker_string = str(ticker_string.lower()).strip()

# # today = date.today()

# # Pass the user input to the yahooquery library
# ticker = get_ticker(ticker_string)

# # Get price history
# stock_df = get_price_hist(ticker=ticker, period=period_option, interval=interval_option)


In [13]:
import os

def save_price_history(ticker_or_df, period_option, interval_option):
    # Get the current working directory
    cwd = os.getcwd()

    # Remove the last folder name from the current working directory to get the path to the parent folder
    path = cwd[:-len(os.path.basename(cwd))]

    # Define the path to daily or weekly stock data
    if interval_option == '1wk':
        data_path = path + 'datasets\\weekly\\'
    elif interval_option == '1d':
        data_path = path + 'datasets\\daily\\'
    else:
        print("Invalid interval option. Please choose '1wk' or '1d'.")
        return

    # Check if the input is a single ticker value or a dataframe
    if isinstance(ticker_or_df, str):
        # Single ticker value
        ticker_string = ticker_or_df.lower().strip()
        ticker = get_ticker(ticker_string)
        file_name = ticker_string + '.csv'
        file_path = data_path + file_name

        # Get price history
        stock_df = get_price_hist(ticker=ticker, period=period_option, interval=interval_option)

        # Save price history as CSV file
        stock_df.to_csv(file_path, index=False)
        print(f"Price history for {ticker_string} saved as {file_name}.")
    elif isinstance(ticker_or_df, pd.DataFrame):
        # Dataframe
        tickers = ticker_or_df['Holding Ticker'].tolist()

        for ticker_string in tickers:
            ticker_string = ticker_string.lower().strip()
            ticker = get_ticker(ticker_string)
            file_name = ticker_string + '.csv'
            file_path = data_path + file_name

            # Get price history
            stock_df = get_price_hist(ticker=ticker, period=period_option, interval=interval_option)

            # Save price history as CSV file
            stock_df.to_csv(file_path, index=False)
            print(f"Price history for {ticker_string} saved as {file_name}.")
    else:
        print("Invalid input. Please provide a single ticker value or a dataframe.")


In [14]:
# Test function with single ticker input
save_price_history(ticker_or_df='AAPL', period_option='5y', interval_option='1wk')

Price history for aapl saved as aapl.csv.


In [17]:
# Test function with dataframe input

# Define holdings dataframe
holdings_df = pd.read_csv(path + 'datasets\\holdings.csv')

save_price_history(ticker_or_df=holdings_df, period_option='5y', interval_option='1wk')

Price history for acic saved as acic.csv.
Price history for amr saved as amr.csv.
Price history for caba saved as caba.csv.
Price history for hnrg saved as hnrg.csv.
Price history for mod saved as mod.csv.
Price history for cbay saved as cbay.csv.
Price history for nog saved as nog.csv.
Price history for dfin saved as dfin.csv.
Price history for lmb saved as lmb.csv.
Price history for trns saved as trns.csv.
Price history for nuvl saved as nuvl.csv.
Price history for px saved as px.csv.
Price history for olma saved as olma.csv.
Price history for cvna saved as cvna.csv.
Price history for mltx saved as mltx.csv.
Price history for coop saved as coop.csv.
Price history for kai saved as kai.csv.
Price history for tdw saved as tdw.csv.
Price history for strl saved as strl.csv.
Price history for kntk saved as kntk.csv.
Price history for spxc saved as spxc.csv.
Price history for dakt saved as dakt.csv.
Price history for gne saved as gne.csv.
Price history for bvh saved as bvh.csv.
Price histor

In [28]:
# Define a list of all the daily data files

# Define the path to daily stock data
daily_data_path = path + 'datasets\\daily\\'

daily_data_files = os.listdir(daily_data_path)

# Define a list of all the weekly data files

# Define the path to weekly stock data
weekly_data_path = path + 'datasets\\weekly\\'

weekly_data_files = os.listdir(weekly_data_path)



# Process all the stock data files
for filename in weekly_data_files[:10]:
    df = pd.read_csv(weekly_data_path + filename)
    
    if is_consolidating(df, percentage=2.5, num_candlesticks=15):
        print("{} is consolidating".format(filename))
    if is_breaking_out(df, percentage=2.5, num_candlesticks=15):
        print("{} is breaking out".format(filename))


In [29]:
candlestic = plot_candlestick(df, ma=False)
candlestic.show()

In [222]:
# cross = ta.cross(stock_df['EMA_50'], stock_df['EMA_100'], above=True)

# # check where the cross happened
# cross[cross == True]

# Extract Fundamental Data from Yahoo Finance

In [ ]:
# check if ticker is a ETF or stock
security_type = ticker.price[ticker_string]['quoteType']
if security_type == 'ETF':
    stock = False
else:
    stock = True




if stock:
    name = ticker.price[ticker_string]['shortName']
    sector = ticker.summary_profile[ticker_string]['sector']
    industry = ticker.summary_profile[ticker_string]['industry']
    holdings = 'NA'
    summary = ticker.summary_profile[ticker_string]['longBusinessSummary']
        # employees = ticker.summary_profile[ticker_string]['fullTimeEmployees']
        # country = ticker.summary_profile[ticker_string]['country']
        # city = ticker.summary_profile[ticker_string]['city']
        # website = ticker.summary_profile[ticker_string]['website']

    # create a dictionary with the required keys and values
    fundamental_data = {'Name': [name], 'Sector': [sector], 'Industry': [industry], 'Summary': [summary]}

    # pass the dictionary to the pandas DataFrame constructor
    fundamental_df = pd.DataFrame(fundamental_data)

    # display fundamental data
    display(fundamental_df)
else:
    pass

,Name,Sector,Industry,Summary
0,"MYR Group, Inc.",Industrials,Engineering & Construction,"MYR Group Inc., through its subsidiaries, prov..."


In [ ]:

if stock:
    # This is a summary of the company's financial data
    valuation_df = ticker.valuation_measures
    # Restrict valuation_df to periodType = 'TTM'
    valuation_df = valuation_df[valuation_df['periodType'] == 'TTM']
    financial_dict = ticker.financial_data[ticker_string]
    financial_df = pd.DataFrame.from_dict(financial_dict, orient='index').reset_index().rename(columns={'index': 'Metric', 0: 'Value'})

    # # More detailed financial data for the company
    
    # income_df = ticker.income_statement()
    # balance_df = ticker.balance_sheet()
    # cash_df = ticker.cash_flow()
   

else:
    holdings = ticker.fund_holding_info[ticker_string]['holdings']
    top_holdings_df = pd.DataFrame(holdings)
    # Rename the columns
    top_holdings_df.columns = ['Symbol','Name', 'Percent']
    # Express the holdings column as a percentage
    top_holdings_df['Percent'] = top_holdings_df['Percent'].apply(lambda x: x * 100)
    # Add a column to the dataframe with today's date
    top_holdings_df['Date'] = today
    # # Display the dataframe
    # display(top_holdings_df)



In [ ]:
# balance_df.reset_index(inplace=True)
# # Convert asOfDate column to year only
# balance_df['asOfDate'] = balance_df['asOfDate'].dt.year
# # Rename asOfDate column to year
# balance_df.rename(columns={'asOfDate': 'Year'}, inplace=True) 

# rev_df = get_financials(df=income_df, col_name='TotalRevenue', metric_name='Total Revenue')
# # Convert Year column to year only
# rev_df['Year'] = rev_df['Year'].dt.year 

# marketcap_df = get_financials(df=valuation_df, col_name='MarketCap', metric_name='Market Cap') # This is updated quarterly

# ebitda_df = get_financials(df=income_df, col_name='NormalizedEBITDA', metric_name='EBITDA')
# # Convert Year column to year only
# ebitda_df['Year'] = ebitda_df['Year'].dt.year


# rev_fig = bar_chart(df=rev_df, x='Year', y='Total Revenue', title='Total Revenue USD')
# rev_fig.show()

# marketcap_fig = line_chart(df=marketcap_df, x='Year', y='Market Cap', title='Market Cap USD')
# marketcap_fig.show()


# ebitda_fig = bar_chart(df=ebitda_df, x='Year', y='EBITDA', title='EBITDA USD')
# ebitda_fig.show()